In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import sklearn
import random
import math
import time

from sklearn.decomposition import PCA

from sp_modif.model_function import *
from sp_modif.methods import *
from sp_modif.data_prep import *
from sp_modif.SHAP import *
from sp_modif.L2X import *
from methods import *

%matplotlib inline
warnings.filterwarnings('ignore')

SEED = 0
def set_seed(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

# Appeler la fonction pour fixer le seed
set_seed(SEED)

In [2]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD004.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc', 'epr','Ps30','phi',
                    'NRf','NRc','BPR', 'farB','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 120
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(61249, 27) (41214, 26) (248, 1)


In [10]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD002.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc', 'epr','Ps30','phi',
                    'NRf','NRc','BPR','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(53759, 27) (33991, 26) (259, 1)


In [11]:
## Data cleaning
alpha = 0.4
# X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha, drop = False)
X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, sensor_names, alpha, drop = False)
X_train_interim.head()

,Unit,Cycle,Altitude,Mach,TRA,T20,T24,T30,T50,P20,...,NRc,BPR,farB,htBleed,Nf_dmd,PCNfR_dmd,W31,W32,RUL,op_cond
0,1,1,35.0,0.84,100.0,0.0,0.325733,0.271554,0.561254,0.0,...,0.052796,0.497413,0.0,0.400000,0.0,0.0,0.373494,0.323963,125,35.0_0.84_100.0
1,1,2,42.0,0.84,100.0,0.0,0.454811,0.362907,0.465316,0.0,...,0.097789,0.453867,0.0,0.358333,0.0,0.0,0.305290,0.395882,125,42.0_0.84_100.0
2,1,3,25.0,0.62,60.0,0.0,0.569392,0.357376,0.462687,0.0,...,0.113134,0.408019,0.0,0.481633,0.0,0.0,0.281380,0.549673,125,25.0_0.62_60.0
3,1,4,42.0,0.84,100.0,0.0,0.494511,0.393808,0.442924,0.0,...,0.108577,0.373161,0.0,0.362418,0.0,0.0,0.368643,0.659291,125,42.0_0.84_100.0
4,1,5,25.0,0.62,60.0,0.0,0.534967,0.386837,0.454447,0.0,...,0.104716,0.372701,0.0,0.465464,0.0,0.0,0.345206,0.551749,125,25.0_0.62_60.0


In [12]:
import pandas as pd
from sklearn.datasets import load_diabetes
from IDFC import IDFC
from utils import compute_variable_correlations

In [13]:
X = X_train_interim[remaining_sensors]

In [14]:
model = IDFC(rho=0.6, verbose=True)
model.fit(X)

Étape 1 : Initialisation (VARCLUS)...
cluster_0 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
cluster_1 [10, 11, 12, 13, 14]
cluster_2 [15]
{'cluster_0': Index(['T24', 'T30', 'T50', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi'], dtype='object'), 'cluster_1': Index(['NRf', 'NRc', 'BPR', 'htBleed', 'W31'], dtype='object'), 'cluster_2': Index(['W32'], dtype='object')}
Étape 2 : Raffinement (CLV K+1)...
Étape 3 : Sélection des variables interprétables...


In [15]:
model.get_selected_features()

['T50']